#### Test on convert a netCDF file to a GeoTIFF

#### To test then in a Leaflet map

 * https://github.com/stuartmatthews/leaflet-geotiff
 * https://mygeodata.cloud/converter/

In [17]:
import numpy as np
import netCDF4
from osgeo import osr
from osgeo import gdal

In [18]:
# 
import os
'GDAL_DATA' in os.environ 

True

In [19]:
# temp.nc has been produced from levitus_climatology with ferret
# yes? use levitus_climatology
# yes? save/clobber/file=temp.nc temp[k=1]

infile = 'temp.nc'

ncfile = netCDF4.Dataset(infile, 'r')
ncfile.variables.keys()

[u'XAXLEVITR', u'YAXLEVITR', u'ZAXLEVITR1_1', u'ZAXLEVITR1_1_bnds', u'TEMP']

In [20]:
dims = ncfile.dimensions.keys()
dims

[u'XAXLEVITR', u'YAXLEVITR', u'ZAXLEVITR1_1', u'bnds']

In [21]:
lon = ncfile.variables[dims[0]][:]
lat = ncfile.variables[dims[1]][:]
var = ncfile.variables['TEMP'][0][:]

print var.shape

(180, 360)


In [22]:
nrows,ncols = np.shape(var)
xres = 360./ncols
yres = 180./nrows

# Retrieve edges considering regular axis
xmin,xmax,ymin,ymax = [lon[0]-xres/2,lon[-1]+xres/2,lat[0]-yres/2,lat[-1]+yres/2]

print nrows,ncols
print xmin,xmax,ymin,ymax
print xres, yres

180 360
20.0 380.0 -90.0 90.0
1.0 1.0


In [24]:
# https://stackoverflow.com/questions/43254024/extracting-specific-netcdf-info-and-converting-to-geotiff-in-python

In [37]:
outfile = 'temp.tif'

driver = gdal.GetDriverByName('GTiff')

dst_ds = driver.Create(outfile, ncols, nrows, 
                       bands=1, eType=gdal.GDT_Float32, options=['COMPRESS=LZW'])

geotransform=(xmin,xres,0,ymax,0,-yres)   
# That's (top left x, w-e pixel resolution, rotation (0 if North is up), 
#         top left y, rotation (0 if North is up), n-s pixel resolution)

dst_ds.SetGeoTransform(geotransform)     # specify coords
srs = osr.SpatialReference()             # establish encoding
srs.ImportFromEPSG(4326)                 # WGS84 lat/long
dst_ds.SetProjection(srs.ExportToWkt())   # export coords to file

dst_ds.SetMetadata({'key1': '1', 'key2': 'yada'})

band = dst_ds.GetRasterBand(1)
band.WriteArray(var[::-1,:])            # reverse the array

band.SetDescription('blabla')
band.SetMetadata({'name': 'AAAAAA', 'key2': 'BBBBB'})
band.SetNoDataValue(-1E10)
                
# Close properly the dataset
dst_ds = None

In [26]:
test1 = osr.SpatialReference()
test1.ImportFromEPSG(4326)

print test1.ExportToPrettyWkt()

GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]


In [29]:
# Check with
! gdalinfo -mm temp.tif

Driver: GTiff/GeoTIFF
Files: temp.tif
Size is 360, 180
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
Origin = (20.000000000000000,90.000000000000000)
Pixel Size = (1.000000000000000,-1.000000000000000)
Metadata:
  AREA_OR_POINT=Area
  key1=1
  key2=yada
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (  20.0000000,  90.0000000) ( 20d 0' 0.00"E, 90d 0' 0.00"N)
Lower Left  (  20.0000000, -90.0000000) ( 20d 0' 0.00"E, 90d 0' 0.00"S)
Upper Right (     380.000,      90.000) (Invalid angle, 90d 0' 0.00"N)
Lower Right (     380.000,     -90.000) (Invalid angle, 90d 0' 0.00"S)
Center      (     200.000,       0.000) (200d 0' 0.00"E,  0d 0' 0.01"N)
Band 1 Block=360x5 Type=Float32, ColorInterp=Gray
  Description = blabla